# Training a CNN for Image-Regression to predict the SDAS of a given image

In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np        #library for matrices (useful for data preparation)
import os                 #library for package manage
import cv2                #library for loading images
import tensorflow as tf   #library/framework for all kinds of machine learning tasks
import matplotlib.pyplot as plt #libary for visualisation
import random             #for random seeds

import tensorflow as tf
from tensorflow.keras.models import Sequential #"It was trained with the Keras library"
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.initializers import Constant, he_normal

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator #for Augmentation of Training Data

from tqdm.keras import TqdmCallback # for loading-bar
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

2025-05-09 08:39:14.692894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746779954.707366 4159182 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746779954.711943 4159182 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746779954.724304 4159182 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746779954.724316 4159182 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746779954.724318 4159182 computation_placer.cc:177] computation placer alr

KeyboardInterrupt: 

In [ ]:
!nvidia-smi

In [ ]:
# speicher test 

In [ ]:
# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
gpus
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

fix random seed. Always use a fixed random seed to guarantee that when you run the code twice you will get the same outcome. This removes a factor of variation and will help keep you sane.

In [ ]:
# 1. Python-Seed für allgemeine Zufallszahlen
random.seed(42)

# 2. NumPy-Seed für alle Operationen mit NumPy
np.random.seed(42)

# 3. TensorFlow-Seed für TensorFlow-Operationen
tf.random.set_seed(42)

# 4. Optional: OS-Level Seed für bestimmte Bibliotheken
os.environ['PYTHONHASHSEED'] = '42'

## Model Architecture

In this part we will first define the model architecture of the CNN, we will train later. The implemented structure is directly copied from this study, which also used a CNN to predict SDAS: https://www.mdpi.com/2075-4701/11/5/756 

*In the present case, the CNN model is based on a basic feedforward (sequential) network.*  

In [ ]:
#Define the model
model = Sequential() 

*The model consists of three 2D convolutional layer blocks. Each convolutional filter is followed by a Rectified Linear Unit (ReLU) activation, batch normalization and a max-pooling layer. Filter sizes used were (5,5)×32, (3,3)×32 and (2,2)×32 for the first, second and third convolutional layers, respectively. Zero-padding was applied evenly across both dimensions to compensate for edges. The convolutions were performed with stride 1. Each max-pooling filter was of size (2,2), and pooling was performed with stride 2. This resulted in the following intermediate activation maps: 200×200×1 (input), 100×100×32 (following the first convolutional block), 50×50×32 (following the second convolutional block) and 25×25×32 (following the third convolutional block).*

## **Explanation Block 1:**

**Convolutional Layer:** Takes the 200x200x1 image as input in the form of a NumPy array and applies 32 different 5x5 kernels/matrices that slide over the input image. For each 5x5 region, it performs simple multiplication and addition to compute a value. The weights in this layer are the values of the kernel matrices, and they are trained during backpropagation. The resulting output is passed through the ReLU (Rectified Linear Unit) activation function, which sets all negative values to 0 and leaves positive values unchanged. To ensure the width and height of the output do not change compared to the input, zero-padding is used. Without zero-padding, each filter would produce a 196x196 output, but with padding, the output remains at 200x200 per filter, by padding the border rows and columns with zeros.

Trainable parameters: For each filter of size 5x5, we need to train 32×5×5 + 32 (biases) = 832 parameters.

**Batch Normalization Layer:** This layer takes the 32 feature maps of size 200×200 from the previous convolutional layer as input. Batch normalization normalizes the inputs per feature map to have a consistent distribution (approximately a normal distribution with mean 0 and variance 1). Why? This helps the training process by reducing the vanishing gradient effect and making convergence more stable.

Trainable parameters:

For each of the 32 feature maps, there are two trainable parameters: γ (scale) and 𝛽 (shift). That results in 32 × 2 = 64 trainable parameters.
Additionally, the layer stores the mean and variance for the inference phase, which accounts for another 64 non-trainable values.
Therefore, the model summary will show 128 parameters for this layer.

**Max Pooling Layer:** After the convolution, a max pooling layer is applied to retain the most important features while reducing the size. A pooling window (in this case 2×2) slides over the image. Within this window, the maximum value is selected. The window moves with a stride of 2, so the output size is reduced by half.

There are no trainable parameters in this layer.

**Reference links:**

Conv: https://www.youtube.com/watch?v=YRhxdVk_sIs&ab_channel=deeplizard

BN: http://youtube.com/watch?v=dXB-KQYkzNU&ab_channel=deeplizard

MP: https://www.youtube.com/watch?v=ZjM_XQa5s6s&ab_channel=deeplizard

In [ ]:
# Block 1
model.add(Conv2D(32, (5,5), strides=1, padding='same', activation='relu', input_shape=(200,200,1))) 
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2), strides=2)) 

# Block 2
model.add(Conv2D(32, (3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

# Block 3
model.add(Conv2D(32, (2,2), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

*Activation of the final convolutional block was then flattened to create a meta-layer of 20,000 neurons, which was then followed by a fully-connected layer consisting of 64 neurons, followed by the ReLU activation function and batch normalization.*

## Explanation:

**Flatten Layer:** Transforms the 32 25x25 feature maps from the last max pooling layer by essentially chaining them together. The result is a one-dimensional vector with 20,000 entries. This layer therefore has 20,000 neurons. There are no trainable weights in this layer.

**Dense Layer:** This is a fully connected layer with 64 neurons, where each neuron is connected to all 20,000 input values. The ReLU activation function is used again.

Trainable parameters: Each of the 64 neurons receives 20,000 input weights, so 20,000 × 64 = 1.28 million parameters, plus 64 biases, resulting in a total of 1,280,064 parameters.

**Batch Normalization Layer:** As in Block 1, the input values are normalized here to stabilize and accelerate the training process.

**Dense Layer:** All 64 normalized input values are connected to a single output neuron.

Trainable parameters: 64 × 1 + 1 = 65 parameters

In [ ]:
# Flatten & Fully Connected Layer
model.add(Flatten())  # 25x25x32 -> 20000 Neuronen
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())

*Finally, this layer was then fully connected to the output layer, which contained a single linear neuron—because of the assumed regression operation. Thus, for a 2D input image of size 200×200×1, a single real value—S value is obtained.*

* **init well**. Initialize the final layer weights correctly. E.g. if you are regressing some values that have a mean of 50 then initialize the final bias to 50. If you have an imbalanced dataset of a ratio 1:10 of positives:negatives, set the bias on your logits such that your network predicts probability of 0.1 at initialization. Setting these correctly will speed up convergence and eliminate “hockey stick” loss curves where in the first few iteration your network is basically just learning the bias.*

In [ ]:
# Output Layer
model.add(Dense(1, activation='linear', kernel_initializer=he_normal(), bias_initializer=Constant(0.4)))

*The Adam optimizer was used, using a learning rate of 0.0005.*

In [ ]:
learning_rate = 0.000005 #extremely important to find good value!
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Modell kompilieren
model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
model.summary() #The total number of parameters was 1,294,977

## Import Dataset and Preprocessing

In [ ]:
# Path to the folder containing the original images (no subfolders)
data_dir = r'../data/bmw_train'
image_exts = ('.jpeg', '.jpg', '.bmp', '.png')
pixel_per_micrometer = 1/0.728265817023213 # for our training data from bmw
F_value = 1.36/pixel_per_micrometer        # F_value from paper

# List all image filenames
files = [f for f in os.listdir(data_dir) if f.lower().endswith(image_exts)] 

tiles = []
labels = []
tile_size = 200

for idx, file in enumerate(files, 1):
    img_path = os.path.join(data_dir, file)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"⚠️ Could not load {file}, skipping.")
        continue

    # --- Extract label from filename exactly as in the original script ---
    try:
        filename_parts = file.split('_')
        if len(filename_parts) < 2:
            print(f"⚠️ Unexpected filename format: {file}")
            continue
        sdas_value_str = filename_parts[1]   # e.g., '33' from 'AC_33_B2.jpg'
        SDAS = float(sdas_value_str)         # Convert string to float
        s_value = round(SDAS / F_value, 2)  # Apply formula
    except Exception as e:
        print(f"⚠️ Error parsing label from '{file}': {e}")
        continue
        
    norm_label = s_value / 100.0  # Normalize to 0–1 range

    h, w = img.shape

    # Split image into 200×200 tiles
    for y in range(0, h, tile_size):
        for x in range(0, w, tile_size):
            tile = img[y:y+tile_size, x:x+tile_size]
            if tile.shape != (tile_size, tile_size):
                continue  # Skip incomplete tiles at the edges

            # Normalize pixel values and add channel dimension
            tile = tile.astype(np.float16) / 255.0
            tile = tile[..., np.newaxis]  # Shape: (200, 200, 1)

            tiles.append(tile)
            labels.append(norm_label)

            # Apply 90°, 180°, and 270° rotations and store them as additional samples
            # for k in [1, 2, 3]:  # 1 = 90°, 2 = 180°, 3 = 270°
            #     rotated = np.rot90(tile, k)  # Rotate k times 90° counter-clockwise
            #     tiles.append(rotated)
            #     labels.append(norm_label)

    if idx % 50 == 0:
        print(f"⏳ Processed {idx}/{len(files)} images...")

# Convert lists to NumPy arrays
input_data = np.array(tiles, dtype=np.float16)
labels     = np.array(labels, dtype=np.float16)

print(f"✅ Done: {input_data.shape[0]} tiles created, input shape {input_data.shape[1:]}")

Processing file 13983/30685
Processing file 13984/30685
Processing file 13985/30685
Processing file 13986/30685
Processing file 13987/30685
Processing file 13988/30685
Processing file 13989/30685
Processing file 13990/30685
Processing file 13991/30685
Processing file 13992/30685
Processing file 13993/30685
Processing file 13994/30685
Processing file 13995/30685
Processing file 13996/30685
Processing file 13997/30685
Processing file 13998/30685
Processing file 13999/30685
Processing file 14000/30685
Processing file 14001/30685
Processing file 14002/30685
Processing file 14003/30685
Processing file 14004/30685
Processing file 14005/30685
Processing file 14006/30685
Processing file 14007/30685
Processing file 14008/30685
Processing file 14009/30685
Processing file 14010/30685
Processing file 14011/30685
Processing file 14012/30685
Processing file 14013/30685
Processing file 14014/30685
Processing file 14015/30685
Processing file 14016/30685
Processing file 14017/30685
Processing file 1401

Processing file 14283/30685
Processing file 14284/30685
Processing file 14285/30685
Processing file 14286/30685
Processing file 14287/30685
Processing file 14288/30685
Processing file 14289/30685
Processing file 14290/30685
Processing file 14291/30685
Processing file 14292/30685
Processing file 14293/30685
Processing file 14294/30685
Processing file 14295/30685
Processing file 14296/30685
Processing file 14297/30685
Processing file 14298/30685
Processing file 14299/30685
Processing file 14300/30685
Processing file 14301/30685
Processing file 14302/30685
Processing file 14303/30685
Processing file 14304/30685
Processing file 14305/30685
Processing file 14306/30685
Processing file 14307/30685
Processing file 14308/30685
Processing file 14309/30685
Processing file 14310/30685
Processing file 14311/30685
Processing file 14312/30685
Processing file 14313/30685
Processing file 14314/30685
Processing file 14315/30685
Processing file 14316/30685
Processing file 14317/30685
Processing file 1431

Processing file 14581/30685
Processing file 14582/30685
Processing file 14583/30685
Processing file 14584/30685
Processing file 14585/30685
Processing file 14586/30685
Processing file 14587/30685
Processing file 14588/30685
Processing file 14589/30685
Processing file 14590/30685
Processing file 14591/30685
Processing file 14592/30685
Processing file 14593/30685
Processing file 14594/30685
Processing file 14595/30685
Processing file 14596/30685
Processing file 14597/30685
Processing file 14598/30685
Processing file 14599/30685
Processing file 14600/30685
Processing file 14601/30685
Processing file 14602/30685
Processing file 14603/30685
Processing file 14604/30685
Processing file 14605/30685
Processing file 14606/30685
Processing file 14607/30685
Processing file 14608/30685
Processing file 14609/30685
Processing file 14610/30685
Processing file 14611/30685
Processing file 14612/30685
Processing file 14613/30685
Processing file 14614/30685
Processing file 14615/30685
Processing file 1461

Processing file 14881/30685
Processing file 14882/30685
Processing file 14883/30685
Processing file 14884/30685
Processing file 14885/30685
Processing file 14886/30685
Processing file 14887/30685
Processing file 14888/30685
Processing file 14889/30685
Processing file 14890/30685
Processing file 14891/30685
Processing file 14892/30685
Processing file 14893/30685
Processing file 14894/30685
Processing file 14895/30685
Processing file 14896/30685
Processing file 14897/30685
Processing file 14898/30685
Processing file 14899/30685
Processing file 14900/30685
Processing file 14901/30685
Processing file 14902/30685
Processing file 14903/30685
Processing file 14904/30685
Processing file 14905/30685
Processing file 14906/30685
Processing file 14907/30685
Processing file 14908/30685
Processing file 14909/30685
Processing file 14910/30685
Processing file 14911/30685
Processing file 14912/30685
Processing file 14913/30685
Processing file 14914/30685
Processing file 14915/30685
Processing file 1491

Processing file 15175/30685
Processing file 15176/30685
Processing file 15177/30685
Processing file 15178/30685
Processing file 15179/30685
Processing file 15180/30685
Processing file 15181/30685
Processing file 15182/30685
Processing file 15183/30685
Processing file 15184/30685
Processing file 15185/30685
Processing file 15186/30685
Processing file 15187/30685
Processing file 15188/30685
Processing file 15189/30685
Processing file 15190/30685
Processing file 15191/30685
Processing file 15192/30685
Processing file 15193/30685
Processing file 15194/30685
Processing file 15195/30685
Processing file 15196/30685
Processing file 15197/30685
Processing file 15198/30685
Processing file 15199/30685
Processing file 15200/30685
Processing file 15201/30685
Processing file 15202/30685
Processing file 15203/30685
Processing file 15204/30685
Processing file 15205/30685
Processing file 15206/30685
Processing file 15207/30685
Processing file 15208/30685
Processing file 15209/30685
Processing file 1521

...

Processing file 29375/30685
Processing file 29376/30685
Processing file 29377/30685
Processing file 29378/30685
Processing file 29379/30685
Processing file 29380/30685
Processing file 29381/30685
Processing file 29382/30685
Processing file 29383/30685
Processing file 29384/30685
Processing file 29385/30685
Processing file 29386/30685
Processing file 29387/30685
Processing file 29388/30685
Processing file 29389/30685
Processing file 29390/30685
Processing file 29391/30685
Processing file 29392/30685
Processing file 29393/30685
Processing file 29394/30685
Processing file 29395/30685
Processing file 29396/30685
Processing file 29397/30685
Processing file 29398/30685
Processing file 29399/30685
Processing file 29400/30685
Processing file 29401/30685
Processing file 29402/30685
Processing file 29403/30685
Processing file 29404/30685
Processing file 29405/30685
Processing file 29406/30685
Processing file 29407/30685
Processing file 29408/30685
Processing file 29409/30685
Processing file 2941

Processing file 29669/30685
Processing file 29670/30685
Processing file 29671/30685
Processing file 29672/30685
Processing file 29673/30685
Processing file 29674/30685
Processing file 29675/30685
Processing file 29676/30685
Processing file 29677/30685
Processing file 29678/30685
Processing file 29679/30685
Processing file 29680/30685
Processing file 29681/30685
Processing file 29682/30685
Processing file 29683/30685
Processing file 29684/30685
Processing file 29685/30685
Processing file 29686/30685
Processing file 29687/30685
Processing file 29688/30685
Processing file 29689/30685
Processing file 29690/30685
Processing file 29691/30685
Processing file 29692/30685
Processing file 29693/30685
Processing file 29694/30685
Processing file 29695/30685
Processing file 29696/30685
Processing file 29697/30685
Processing file 29698/30685
Processing file 29699/30685
Processing file 29700/30685
Processing file 29701/30685
Processing file 29702/30685
Processing file 29703/30685
Processing file 2970

Processing file 29970/30685
Processing file 29971/30685
Processing file 29972/30685
Processing file 29973/30685
Processing file 29974/30685
Processing file 29975/30685
Processing file 29976/30685
Processing file 29977/30685
Processing file 29978/30685
Processing file 29979/30685
Processing file 29980/30685
Processing file 29981/30685
Processing file 29982/30685
Processing file 29983/30685
Processing file 29984/30685
Processing file 29985/30685
Processing file 29986/30685
Processing file 29987/30685
Processing file 29988/30685
Processing file 29989/30685
Processing file 29990/30685
Processing file 29991/30685
Processing file 29992/30685
Processing file 29993/30685
Processing file 29994/30685
Processing file 29995/30685
Processing file 29996/30685
Processing file 29997/30685
Processing file 29998/30685
Processing file 29999/30685
Processing file 30000/30685
Processing file 30001/30685
Processing file 30002/30685
Processing file 30003/30685
Processing file 30004/30685
Processing file 3000

Processing file 30270/30685
Processing file 30271/30685
Processing file 30272/30685
Processing file 30273/30685
Processing file 30274/30685
Processing file 30275/30685
Processing file 30276/30685
Processing file 30277/30685
Processing file 30278/30685
Processing file 30279/30685
Processing file 30280/30685
Processing file 30281/30685
Processing file 30282/30685
Processing file 30283/30685
Processing file 30284/30685
Processing file 30285/30685
Processing file 30286/30685
Processing file 30287/30685
Processing file 30288/30685
Processing file 30289/30685
Processing file 30290/30685
Processing file 30291/30685
Processing file 30292/30685
Processing file 30293/30685
Processing file 30294/30685
Processing file 30295/30685
Processing file 30296/30685
Processing file 30297/30685
Processing file 30298/30685
Processing file 30299/30685
Processing file 30300/30685
Processing file 30301/30685
Processing file 30302/30685
Processing file 30303/30685
Processing file 30304/30685
Processing file 3030

Processing file 30564/30685
Processing file 30565/30685
Processing file 30566/30685
Processing file 30567/30685
Processing file 30568/30685
Processing file 30569/30685
Processing file 30570/30685
Processing file 30571/30685
Processing file 30572/30685
Processing file 30573/30685
Processing file 30574/30685
Processing file 30575/30685
Processing file 30576/30685
Processing file 30577/30685
Processing file 30578/30685
Processing file 30579/30685
Processing file 30580/30685
Processing file 30581/30685
Processing file 30582/30685
Processing file 30583/30685
Processing file 30584/30685
Processing file 30585/30685
Processing file 30586/30685
Processing file 30587/30685
Processing file 30588/30685
Processing file 30589/30685
Processing file 30590/30685
Processing file 30591/30685
Processing file 30592/30685
Processing file 30593/30685
Processing file 30594/30685
Processing file 30595/30685
Processing file 30596/30685
Processing file 30597/30685
Processing file 30598/30685
Processing file 3059

In [ ]:
print(input_data.shape)  # (n, 200, 200, 1) 
print(labels.shape)      # (n,) 

In [ ]:
type(input_data)

In [ ]:
type(labels)

In [ ]:
def check_duplicates_within_set(X, Y):
    unique_X, unique_indices = np.unique(X, axis=0, return_index=True)
    unique_Y = Y[unique_indices]

    print(f"{len(X)-len(unique_X)} duplicates found.")

    return unique_X, unique_Y

In [ ]:
X, Y = check_duplicates_within_set(input_data, labels)

In [ ]:
print(X.shape)  # (n, 200, 200, 1) 
print(Y.shape)  # (n,) 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.4, random_state=42) # stratify with labels (Y)

In [ ]:
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)

In [ ]:
print(X_train.shape)  # (n, 200, 200, 1)
print(X_test.shape)   # (n, 200, 200, 1)
print(X_val.shape)    # (n, 200, 200, 1)

In [ ]:
print(Y_train.shape)  # (n,) 
print(Y_test.shape)   # (n,) 
print(Y_val.shape)    # (n,) 

*The training set, validation set and test set were split in the ratio 60 : 20 : 20.* 

## Augmentation

In [ ]:
def custom_rotation(x):
    angle = np.random.choice([90, 180, 270])  # Randomly selects between 90, 180, and 270 degrees
    return np.rot90(x, k=angle // 90)  # k specifies the number of 90° rotations

# Define data augmentation (only for training data)
datagen = ImageDataGenerator(
    preprocessing_function=custom_rotation,  # Limit rotations to 90°, 180°, and 270°
    horizontal_flip=True,   # Flip images horizontally
)

# Fit the generator to the training data
datagen.fit(X_train)

## Data Understanding 

### Check if x any y fit together

In [ ]:
X_train[1].shape

In [ ]:
type(X_train[1])

In [ ]:
# Convert to float32
image = X_train[1].squeeze().astype(np.float32)

# Plot the image
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
Y_train[1]

### Check histogramm for image values

In [ ]:
plt.hist(X_train[1].flatten(), bins=50, range=(0, 1))
plt.title('Verteilung der Pixelwerte')
plt.show()

In [ ]:
print(f"Min: {np.min(X_train)}, Max: {np.max(X_train)}")

### Check if null or inf values in train_y

In [ ]:
assert not np.any(np.isnan(image)), "NaN-values found"
assert not np.any(np.isinf(image)), "infinite values found"
assert X_train[5].shape == (200, 200, 1), "image has unexpected form"

### Check Data Distribution

In [ ]:
# Plot histogram
plt.hist(Y_train, bins=100, edgecolor='black')  # 100 bins, black edges
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Histogram of Train")
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Plot histogram
plt.hist(Y_test, bins=100, edgecolor='black')  # 100 bins, black edges
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Histogram of Test")
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Plot histogram
plt.hist(Y_val, bins=100, edgecolor='black')  # 100 bins, black edges
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Histogram of Val")
plt.grid(True)

# Show the plot
plt.show()

## Training

In [ ]:
logdir = r'..\logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    r'..\checkpoints\best_weights.weights.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None
)

*It was trained with the Keras library on an Intel® Core™ I7-4970 CPU running at 3.60 GHz using 8 parallel processing units. Training was completed in 5 h—for 30 epochs total. A batch size of 32 was used for the training.* 

I will instead use my GPU, since its simply way faster.

In [ ]:
# Check if GPU's are available
gpus = tf.config.list_physical_devices('GPU')
print(f"{len(gpus)} GPUs erkannt:")
for gpu in gpus:
    print(gpu)

In [ ]:
# If a GPU is available, enable memory growth (prevents memory issues)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        print(e)

*Verify loss @ init. Verify that your loss starts at the correct loss value*

In [ ]:
# Calculate the initial loss before training
init_loss = model.evaluate(X_val, Y_val, verbose=1)
print(f"Initial Loss: {init_loss[0]}")  # Should be approximately 1/3

In [ ]:
# CALLBACKS (optional)

# EarlyStopping callback, stops training when validation loss does not improve
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# ModelCheckpoint callback, saves the best model based on validation loss
# checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

def scheduler(epoch, lr): # Exponential Decay
    decay_rate = 0.90     # reduced 10% every decay_epochs...
    decay_epoch = 100     
    return lr * decay_rate**(epoch // decay_epoch)

lr_scheduler = LearningRateScheduler(scheduler)

# HYPERPARAMETER

batch_size = 32    # the original paper used a batch size of 32
epochs = 1000       # the original paper used 30 epochs 
shuffle = True

hist = model.fit(
    datagen.flow(X_train, Y_train, batch_size=batch_size),  # Use the generator here
    epochs=epochs,
    validation_data=(X_val, Y_val),
    shuffle=shuffle,
    verbose=0,  # muss 0 sein für tqdm
    callbacks=[TqdmCallback(verbose=1), early_stopping, lr_scheduler]
)

## Plot Train, Test and Val Loss

*add significant digits to your eval. When plotting the test loss run the evaluation over the entire (large) test set. Do not just plot test losses over batches and then rely on smoothing them in Tensorboard. We are in pursuit of correctness and are very willing to give up time for staying sane.*

In [ ]:
# Evaluation on the entire test set
test_loss = model.evaluate(X_test, Y_test, batch_size = 1, verbose=1)

In [ ]:
print(hist.history.keys())

In [ ]:
print(hist.history)

In [ ]:
# Mean Squared Errors (Loss)
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='train_loss', marker='.')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss', marker='.')
plt.axhline(y=test_loss[0], color='gray', linestyle='--', label='test_loss (final)')
plt.ylim(0, 0.1) # try commenting this...
fig.suptitle('Loss = Mean Squared Error', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# # Root Mean Squared Errors (Metric) 
fig = plt.figure()
plt.plot(hist.history['root_mean_squared_error'], color='teal', label='train_rmse', marker='.') # depends on tensorflow version if root_mean_squared_error or RootMeanSquaredError
plt.plot(hist.history['val_root_mean_squared_error'], color='orange', label='val_rmse', marker='.')
plt.axhline(y=test_loss[1], color='gray', linestyle='--', label='test_rmse (final)')
plt.ylim(0, 1)
fig.suptitle('Root Mean Squared Error', fontsize=20)
plt.legend(loc="upper left")
plt.show()

## Evaluate Performance

The labels from the paper are in a range from 0.06 to 0.65.

### Performance

In [ ]:
mse = MeanSquaredError()
rmse = RootMeanSquaredError()
mape = MeanAbsolutePercentageError()
mae = MeanAbsoluteError()

In [ ]:
batch_size = 1  # If your model was trained with batches (for a more accurate test, it's still better to use 1)

y_true = []
y_pred = []

# Calculate metrics over all test data in batches
for i in range(0, len(X_test), batch_size):
    X_batch = X_test[i:i+batch_size]  # Take a batch of test data
    y_batch = Y_test[i:i+batch_size]  # Corresponding labels
    yhat_batch = model.predict(X_batch, verbose=0)  # Model prediction for the batch
    
    # Store true values and predictions
    y_true.append(y_batch)
    y_pred.append(yhat_batch)

    # Update metrics
    mse.update_state(y_batch, yhat_batch)
    rmse.update_state(y_batch, yhat_batch)
    mae.update_state(y_batch, yhat_batch)
    mape.update_state(y_batch, yhat_batch)
    
y_true = np.concatenate(y_true, axis=0)
y_pred = np.concatenate(y_pred, axis=0)

# R²-Score
r2 = r2_score(y_true, y_pred)

In [ ]:
print(f"Mean Squared Error (MSE): {mse.result().numpy():.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse.result().numpy():.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape.result().numpy():.2f}%")
print(f"Mean Absolute Error (MAPE): {mae.result().numpy():.2f}")
print(f"R²-Score: {r2}")

## Predict SDAS for some specific test image

In [ ]:
test_index = 1001
test_image = X_test[test_index]
test_label = Y_test[test_index]

print(test_image.shape)

# Convert Image to float32 numpy array
image = test_image.squeeze().astype(np.float32)

# Plot the image
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
print(np.expand_dims(test_image, axis=0).shape)

In [ ]:
yhat = model.predict(np.expand_dims(test_image, axis=0))

In [ ]:
print("Ground Truth S_value: ", test_label*100)
print("Predicted S_value: ", round(yhat[0][0] * 100, 2))

## Export Model

In [ ]:
# model.save(os.path.join(r'..\models', '29.4.25_v2.keras')) 

## Make Predictions on real images

In [ ]:
#test_dataset_version = "bmw_data" #210 images

test_dir = r'../data/bmw_test' 

In [ ]:
min_size = 200

results = []

# for all images in test_dir
for idx, img_name in enumerate(os.listdir(test_dir)):
    if img_name.endswith('.jpg') or img_name.endswith('.png'):
        
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        height, width = img.shape
        
        # Calculate the number of tiles
        rows = height // min_size
        cols = width // min_size
        
        # Split the image into tiles
        images = []
        for i in range(rows):
            for j in range(cols):
                y_start = i * min_size
                x_start = j * min_size
                y_end = min(y_start + min_size, height)
                x_end = min(x_start + min_size, width)
                image = img[y_start:y_end, x_start:x_end]
                images.append(image)
        
        # Collect model predictions
        predictions = []
        for image in images:
            image = image.astype(np.float16) / 255.0  # normalize to [0, 1]
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))  # add channel dimension            
            prediction = model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            predictions.append(prediction)
        
        # Calculate the prediction
        S_value_pred = np.median(predictions) * 100 # * 100, because we normalized the data when we imported it 
        
        SDAS_pred = S_value_pred * F_value  
        
        # Extract the actual SDAS value from the filename
        try:
            SDAS_true = float(img_name.split('_')[1])
            results.append((SDAS_true, SDAS_pred))
        except:
            print(f"Warning: Couldn't extract SDAS-value from '{img_name}'!")

        if idx % 5 == 0:
            print(f"⏳ Processed {idx}/{len(os.listdir(test_dir))} images...")

In [ ]:
# Separate true and predicted values
y_true = [true for true, pred in results]
y_pred = [pred for true, pred in results]

# Calculate the metrics
SDAS_mse = mean_squared_error(y_true, y_pred)
SDAS_rmse = np.sqrt(SDAS_mse)
SDAS_mae = mean_absolute_error(y_true, y_pred)
SDAS_r2 = r2_score(y_true, y_pred)

# MAPE (with protection against division by zero)
y_true_array = np.array(y_true)
y_pred_array = np.array(y_pred)
SDAS_mape = np.mean(np.abs((y_true_array - y_pred_array) / y_true_array)) * 100

# Output the results
print(f"MSE: {SDAS_mse:.4f}")
print(f"RMSE: {SDAS_rmse:.4f}")
print(f"MAE: {SDAS_mae:.4f}")
print(f"MAPE: {SDAS_mape:.2f}%")
print(f"R²: {SDAS_r2:.4f}")

In [ ]:
len(y_true_array)

In [ ]:
#plotting predicted vs ground truth SDAS values for all images

plt.figure(figsize=(6, 6))  
plt.plot([0, 100], [0, 100], 'r--')  # Reference diagonal line
plt.scatter(y_true_array, y_pred_array)
plt.xlabel("Ground Truth SDAS")
plt.ylabel("Predicted SDAS")
plt.axis("equal")  # Ensure equal scaling on both axes
plt.title("Predicted vs. Ground Truth SDAS")
plt.show()

## Showing Sclices Predictions for an given image

In [ ]:
def process_image(test_dir, img_name, model):
    img_path = os.path.join(test_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape
    
    # Calculate the number of tiles
    rows = height // min_size
    cols = width // min_size
    
    # Split the image into tiles
    images = []
    predictions = []
    for i in range(rows):
        for j in range(cols):
            y_start = i * min_size
            x_start = j * min_size
            y_end = min(y_start + min_size, height)
            x_end = min(x_start + min_size, width)
            image = img[y_start:y_end, x_start:x_end]
            images.append(image)
            
            # Normalize and resize the image for prediction
            image = image.astype(np.float16) / 255.0  # normalize to [0, 1]
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))  # Add channel dimension            
            prediction = model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            predictions.append(prediction * 100 * F_value)
    
    return img, predictions, rows, cols

def plot_heatmap_with_image(img, predictions, rows, cols, min_size):
    # Reshape predictions to match the number of tiles
    heatmap = np.array(predictions).reshape((rows, cols))
    
    # Create a figure
    plt.figure(figsize=(10, 10))
    
    # Display the original image
    plt.imshow(img, cmap='gray', extent=[0, img.shape[1], img.shape[0], 0])
    
    # Overlay the heatmap on top of the image
    plt.imshow(heatmap, cmap='jet', alpha=0.5, extent=[0, img.shape[1], img.shape[0], 0])
    
    # Add color bar (legend for the heatmap)
    cbar = plt.colorbar()
    cbar.set_label('SDAS Prediction')

    # Show the plot
    plt.title("Image with SDAS Heatmap Overlay")
    plt.show()

In [ ]:
test_dir = r'../data/bmw_test'
img_name = 'AC_39_B9.jpg'  # Replace with the image you want to test

# Process the selected image
img, predictions, rows, cols = process_image(test_dir, img_name, model)

In [ ]:
print(np.median(predictions))

# Plot the heatmap over the image
plot_heatmap_with_image(img, predictions, rows, cols, min_size)

## ML-Flow 

In [ ]:
!pip install --user mlflow
!pip install mlflow

In [ ]:
import mlflow
import mlflow.keras
from mlflow.tracking import MlflowClient

Tutorial: https://www.youtube.com/watch?v=6ngxBkx05Fs&ab_channel=codebasics

What do we want to log for every run:
- model (with summary)
- hyperparameter
- dataset version
- metrics
- notebook

Run "mlflow ui" in your CMD first!

In [ ]:
!mlflow --version

In [ ]:
mlflow.set_experiment("SDAS_Prediciton")
mlflow.set_tracking_uri("file:../mlruns")

with mlflow.start_run(run_name = 'v1_CNN_paperArchitecture_bmw_unaugmented'): # change with every run 
    
    # Save hyperparameters
    
    mlflow.log_params({
        'learning_rate': learning_rate,
        'batch_size': batch_size,
        'epochs': epochs,
        'shuffle': shuffle,
        'optimizer': optimizer,
        # Save Data
        #'dataset_version': dataset_version,  
        #'test_dataset_version': test_dataset_version,
        'pixel_per_micrometer ': pixel_per_micrometer 
    }) 
    
    # Save metrics
    
    mlflow.log_metrics({
        'S_norm_Mean Squared Error': mse.result().numpy(),
        'S_norm_Root Mean Squared Error': rmse.result().numpy(), 
        'S_norm_Mean Absolute Percentage Error': mape.result().numpy(),
        'S_norm_Mean Absolute Error': mae.result().numpy(),
        'S_norm_R2-Score': r2,
        'SDAS_Mean Squared Error': SDAS_mse,
        'SDAS_Root_Mean Squared Error': SDAS_rmse,
        'SDAS_Mean Absolute Percentage Error': SDAS_mape, 
        'SDAS_Mean Absolute Error': SDAS_mae,
        'SDAS_R2-Score': SDAS_r2
    }) 
    
    # Save model
    
    if isinstance(model, tf.keras.Model):
        mlflow.keras.log_model(model, artifact_path="cnn_model")
    else:
        print("The model isn't from the type tf.keras.Model!") 
        
    # Save notebook
    
    mlflow.log_artifact("CNN_SDAS_Prediction_Pipeline.ipynb", artifact_path="notebooks") # Name of notebook has to be correct

## Load model from ML-Flow

In [ ]:
# Enter RunID and Path
run_id = input("Enter your RunID: ")  # Ersetze mit deiner RunID
model_path = "cnn_model"

# load model
loaded_model = mlflow.keras.load_model(f"runs:/{run_id}/{model_path}")

# print model architecture
loaded_model.summary()

In [ ]:
y_pred = loaded_model.predict(X_test)
y_pred[:4]

In [ ]:
# optional

client = MlflowClient()

# load hyperparameter and metrcis
run_data = client.get_run(run_id).data
print("Hyperparameter:", run_data.params)
print("Metriken:", run_data.metrics)

# download current notebook
client.download_artifacts(run_id, f"notebooks_from_mlflow/{run_id}.ipynb", "./")